In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset
from brainstorm_solver import BrainstormSolver
import json
from pathlib import Path
from settings.logger import logger

import logging

logging.basicConfig(level=logging.INFO)  # or DEBUG, WARNING, ERROR
logging.getLogger('httpx').setLevel(logging.WARNING)

c:\Users\ASUS\miniconda3\envs\torchenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv

load_dotenv("env/credentials.env")

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

In [ ]:
ds_train = load_dataset(
    "deepmind/code_contests",
    cache_dir="dataset",
    split='valid'
)

In [5]:
from settings.agent_config import AGENTS
from knowledge_builder import build_all_agent_knowledge

filepaths = build_all_agent_knowledge(
    dataset=ds_train,
    agent_configs=AGENTS,
    output_dir="./agent_knowledge",
    count=10,  # 10 problems per agent
    language=3  # Python
)

print("\nKnowledge files created:")
for name, path in filepaths.items():
    print(f"  {name}: {path}")

INFO:root:======================================================================
INFO:root:BUILDING AGENT KNOWLEDGE BASE
INFO:root:======================================================================
INFO:root:Knowledge already exists for Graph agent at ./agent_knowledge\graph_agent.json
INFO:root:Knowledge already exists for Brute Force agent at ./agent_knowledge\brute_force_agent.json
INFO:root:Knowledge already exists for Data Structures agent at ./agent_knowledge\data_structures_agent.json
INFO:root:Knowledge already exists for Dynamic Programming agent at ./agent_knowledge\dynamic_programming_agent.json
INFO:root:Knowledge already exists for Math agent at ./agent_knowledge\math_agent.json
INFO:root:
INFO:root:KNOWLEDGE BASE BUILD COMPLETE
INFO:root:======================================================================
INFO:root:Total agents: 5
INFO:root:Output directory: ./agent_knowledge
INFO:root:Problems per agent: 10



Knowledge files created:
  Graph agent: ./agent_knowledge\graph_agent.json
  Brute Force agent: ./agent_knowledge\brute_force_agent.json
  Data Structures agent: ./agent_knowledge\data_structures_agent.json
  Dynamic Programming agent: ./agent_knowledge\dynamic_programming_agent.json
  Math agent: ./agent_knowledge\math_agent.json


In [6]:
from settings.agent_config import AGENTS
from rag_agent import MistralRAGAgent
from knowledge_builder import load_agent_knowledge

agent_system = {}
for config in AGENTS:
    name = config["name"]
    expertise = config["expertise"]
    agent = MistralRAGAgent(
        name=name,
        expertise=expertise,
        api_key=MISTRAL_API_KEY,
        qdrant_path=os.path.join("qdrant_storage", name),
        cache_dir=os.path.join("embedding_cache", name)
    )
    
    # Load pre-built knowledge
    knowledge = load_agent_knowledge(name, "agent_knowledge")
    
    print(f"Loading {len(knowledge)} problems for {name}...")
    
    # Add to agent (uses Qdrant + embedding cache)
    for item in knowledge:
        agent.add_knowledge(
            content=item['content'],
            metadata=item['metadata']
        )
        #time.sleep(1) #to avoid rate limits
    
    print(f"✓ {name} ready ({agent.get_knowledge_count()} items)")

    agent_system[name] = agent

Loading 10 problems for Graph agent...
✓ Graph agent ready (144 items)
Loading 10 problems for Brute Force agent...
✓ Brute Force agent ready (140 items)
Loading 10 problems for Data Structures agent...
✓ Data Structures agent ready (140 items)
Loading 10 problems for Dynamic Programming agent...
✓ Dynamic Programming agent ready (140 items)
Loading 10 problems for Math agent...
✓ Math agent ready (140 items)


In [7]:
agent_system

{'Graph agent': <rag_agent.MistralRAGAgent at 0x16f9feed160>,
 'Brute Force agent': <rag_agent.MistralRAGAgent at 0x1701d3fbd90>,
 'Data Structures agent': <rag_agent.MistralRAGAgent at 0x1701d52ad50>,
 'Dynamic Programming agent': <rag_agent.MistralRAGAgent at 0x1701d4b29e0>,
 'Math agent': <rag_agent.MistralRAGAgent at 0x1701d718d60>}

In [9]:
task = ds_train[-2]['description']

solver = BrainstormSolver(agent_system, max_review_rounds=2)
result = solver.solve(task)

INFO:root:======================================================================
INFO:root:PHASE 1 — Confidence Estimation
INFO:root:======================================================================
INFO:root:  Graph agent            0.80  — The problem involves graph theory and matrix manipulation, which are within my expertise.
INFO:root:  Brute Force agent      0.30  — The problem involves a complex matrix and queries, which is not a typical exhaustive search problem.
INFO:root:  Data Structures agent  0.80  — The problem involves designing an efficient data structure to handle queries and maintain the beautiful matrix property, which is within my expertise.
INFO:root:  Dynamic Programming agent 0.30  — The problem involves dynamic programming and game theory, but the constraints and the specific requirements for the beautiful matrix make it complex.
INFO:root:  Math agent             0.30  — The problem involves combinatorics and matrix manipulation, but the solution requires 

In [10]:
serializable_output = {
    "task": task,
    "confidences": result.get("confidences", {}),
    "core_agents": result.get("core_agents", []),
    "ideas": [
        {
            "idea_id": idea.idea_id,
            "text": idea.text,
            "author": idea.author,
            "score": idea.score,
            "supporters": idea.supporters,
            "critics": idea.critics
        }
        for idea in result.get("ideas", [])
    ],
    "solutions": result.get("solutions", {}),
    "votes": result.get("votes", {}),
    "winner": result.get("winner"),
    "final_summary": result.get("final_summary", {})
}

filepath = Path("solution_example.json")
with open(filepath, 'w', encoding='utf-8') as f:
    json.dump(serializable_output, f, indent=2, ensure_ascii=False)

logger.info(f"Solution output saved to {filepath}")

INFO:root:Solution output saved to solution_example.json


In [11]:
print(task)

A matrix of size n × m, such that each cell of it contains either 0 or 1, is considered beautiful if the sum in every contiguous submatrix of size 2 × 2 is exactly 2, i. e. every "square" of size 2 × 2 contains exactly two 1's and exactly two 0's.

You are given a matrix of size n × m. Initially each cell of this matrix is empty. Let's denote the cell on the intersection of the x-th row and the y-th column as (x, y). You have to process the queries of three types:

  * x y -1 — clear the cell (x, y), if there was a number in it; 
  * x y 0 — write the number 0 in the cell (x, y), overwriting the number that was there previously (if any); 
  * x y 1 — write the number 1 in the cell (x, y), overwriting the number that was there previously (if any). 



After each query, print the number of ways to fill the empty cells of the matrix so that the resulting matrix is beautiful. Since the answers can be large, print them modulo 998244353.

Input

The first line contains three integers n, m an

In [15]:
print(result['winner'])

Graph agent


In [16]:
winner = result['winner']
print(result['solutions'][winner]['solution'])

from collections import defaultdict

MOD = 998244353

class SegmentTree:
    def __init__(self, size):
        self.size = size
        self.tree = [0] * (4 * size)

    def update(self, node, start, end, idx, value):
        if start == end:
            self.tree[node] = value
        else:
            mid = (start + end) // 2
            if idx <= mid:
                self.update(2 * node, start, mid, idx, value)
            else:
                self.update(2 * node + 1, mid + 1, end, idx, value)
            self.tree[node] = (self.tree[2 * node] + self.tree[2 * node + 1]) % MOD

    def query(self, node, start, end, l, r):
        if r < start or end < l:
            return 0
        if l <= start and end <= r:
            return self.tree[node]
        mid = (start + end) // 2
        left = self.query(2 * node, start, mid, l, r)
        right = self.query(2 * node + 1, mid + 1, end, l, r)
        return (left + right) % MOD

def solve():
    n, m, k = map(int, input().split())
  